In [280]:
import pandas as pd
import numpy as np
from collections import Counter
from scipy.sparse import csr_matrix, save_npz
from tqdm.notebook import tqdm
from itertools import combinations, product
from pprint import pformat

In [2]:
train = pd.read_json('../data/train.json')

In [121]:
train.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000


In [85]:
songs = pd.read_json('../data/song_meta.json')

In [86]:
songs.head()

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4


## preprocessing - duplicated rows(subset on tags and songs)
* 중복된 플레이리스트 중, like_cnt가 높은 row만 남기기

In [168]:
temp = train.copy()
temp.sort_values(by='like_cnt', ascending=False, inplace=True)
temp['tags'] = temp['tags'].astype('str')
temp['songs'] = temp['songs'].astype('str')

In [170]:
temp[temp.duplicated(subset=['tags','songs'])]

,tags,id,plylst_title,songs,like_cnt,updt_date
48371,['트로트'],64358,트로트1,"[35848, 638330, 132604, 666824, 231868, 546740...",347,2019-10-16 18:36:49.000
92791,['트로트'],151788,트로트1,"[35848, 638330, 132604, 666824, 231868, 546740...",305,2019-10-16 18:48:31.000
41489,['트로트'],69815,트로트1,"[35848, 638330, 132604, 666824, 231868, 546740...",261,2019-10-16 18:38:50.000
66977,['트로트'],131747,트로트1,"[35848, 638330, 132604, 666824, 231868, 546740...",234,2019-10-16 18:35:31.000
95765,['트로트'],125473,트로트1,"[35848, 638330, 132604, 666824, 231868, 546740...",223,2019-10-16 18:58:48.000
...,...,...,...,...,...,...
3791,"['기분전환', '발라드']",135245,크리스마스44,"[405485, 381653, 566122, 546313, 158539, 68697...",0,2019-11-19 11:02:01.000
16678,['뉴에이지'],126891,뉴에이지 음악을 들으며 오후 보내기,"[22390, 653393, 294137, 585016, 192689, 534014...",0,2015-09-10 15:32:36.000
71422,"['기분전환', '잔잔한']",84807,크리스마스312,"[381653, 566122, 546313, 158539, 686976, 14423...",0,2019-11-19 14:16:05.000
71424,"['드라이브', '발라드']",98199,크리스마스 18,"[405485, 381653, 566122, 546313, 158539, 68697...",0,2019-11-18 17:29:48.000


In [171]:
temp[(temp['tags']=="['인디음악']")&(temp['plylst_title']=='인디음악')]

,tags,id,plylst_title,songs,like_cnt,updt_date
68692,['인디음악'],125124,인디음악,"[206346, 634447, 622600, 292011, 418935, 37203...",294,2019-10-16 19:17:02.000
100256,['인디음악'],128649,인디음악,"[206346, 634447, 622600, 292011, 418935, 37203...",180,2019-10-16 19:15:58.000
26501,['인디음악'],146850,인디음악,"[206346, 634447, 622600, 292011, 418935, 37203...",151,2019-10-16 19:17:43.000
69425,['인디음악'],99194,인디음악,"[206346, 634447, 622600, 292011, 418935, 37203...",28,2019-10-16 19:14:30.000


In [172]:
temp[(temp['tags']=="['트로트']")&(temp['plylst_title']=='트로트1')]

,tags,id,plylst_title,songs,like_cnt,updt_date
21608,['트로트'],114007,트로트1,"[35848, 638330, 132604, 666824, 231868, 546740...",567,2019-10-16 18:38:13.000
48371,['트로트'],64358,트로트1,"[35848, 638330, 132604, 666824, 231868, 546740...",347,2019-10-16 18:36:49.000
92791,['트로트'],151788,트로트1,"[35848, 638330, 132604, 666824, 231868, 546740...",305,2019-10-16 18:48:31.000
67611,['트로트'],103057,트로트1,"[546740, 483630, 616798, 453477, 431049, 35848...",266,2019-10-16 18:36:08.000
41489,['트로트'],69815,트로트1,"[35848, 638330, 132604, 666824, 231868, 546740...",261,2019-10-16 18:38:50.000
66977,['트로트'],131747,트로트1,"[35848, 638330, 132604, 666824, 231868, 546740...",234,2019-10-16 18:35:31.000
95765,['트로트'],125473,트로트1,"[35848, 638330, 132604, 666824, 231868, 546740...",223,2019-10-16 18:58:48.000
28166,['트로트'],49039,트로트1,"[35848, 638330, 132604, 666824, 231868, 546740...",214,2019-10-16 18:45:02.000
23304,['트로트'],139554,트로트1,"[35848, 638330, 132604, 666824, 231868, 546740...",188,2019-10-16 18:39:28.000
110945,['트로트'],152039,트로트1,"[35848, 638330, 132604, 666824, 231868, 546740...",109,2019-10-16 18:58:14.000


In [174]:
temp.drop_duplicates(subset=['tags','songs'], keep='first', inplace=True)

In [176]:
nondup_index = temp.index

In [178]:
train = train.iloc[nondup_index, :]

In [179]:
train.shape

(114694, 6)

In [182]:
train.reset_index(drop=True, inplace=True)

## check 

In [183]:
playlists = train.songs.to_list()
tags = train.tags.to_list()

In [251]:
like_cnts = train.like_cnt.to_list()

In [185]:
for i, playlist in enumerate(playlists):
    if 660324 in playlist: print(i)

41621
82391
96563
96968
99473
100935
101221
103735
104747
104842
105085
105240
105315
105838
105983
106281
106454
106511
106675
106986
107118
107425
107702
107826
107962
108118
108364
109023
109642
109670
109721
110140
110401
110444
110450
110473
110620
111047
111181
111731
112107
112732
113314
113521
114071
114162
114622


In [186]:
for i, playlist in enumerate(playlists):
    if 660324 in playlist: print(i)

41621
82391
96563
96968
99473
100935
101221
103735
104747
104842
105085
105240
105315
105838
105983
106281
106454
106511
106675
106986
107118
107425
107702
107826
107962
108118
108364
109023
109642
109670
109721
110140
110401
110444
110450
110473
110620
111047
111181
111731
112107
112732
113314
113521
114071
114162
114622


In [230]:
train[train['like_cnt']==0]

,tags,id,plylst_title,songs,like_cnt,updt_date
104638,"[릴렉스, 야호]",141134,13기다려,"[682184, 381874, 703096, 209782, 628232, 58065...",0,2019-11-18 16:31:37.000
104639,[기분전환],708,더위를 날릴 노래,"[24899, 493804, 625200, 144663, 140920, 648635...",0,2019-04-15 18:07:15.000
104640,"[특별한, 휴가]",26400,휴가 계획은 세우셨나요? 333,"[254721, 673213, 49941, 34985, 521993, 600726,...",0,2019-05-09 13:51:10.000
104641,[록],84060,"한국 Rock, 새로운 감성을 입다 38","[186263, 100699, 242286, 165412, 237051, 28604...",0,2019-03-27 15:28:25.000
104642,[하늘],64337,'하늘' 노래제목 들어간 노래,"[133309, 329389, 283138, 391291, 704025, 64557...",0,2016-04-19 20:51:02.000
...,...,...,...,...,...,...
114689,[어쿠스틱기타],132666,어쿠스틱 좋은 노래,"[110540, 688028, 624009, 593300, 494305, 65325...",0,2018-04-24 19:10:01.000
114690,[락],9179,라디오 797,"[27673, 112731, 550192, 647261, 509152, 314370...",0,2010-11-09 13:28:23.000
114691,"[잔잔한, 힐링, 명상, 피아노, 차분함]",42577,따스한 감성을 전하는 피아노 음악,"[274395, 95894, 201745, 191637, 602182, 200150...",0,2019-05-03 10:14:33.000
114692,"[댄스, 댄스곡, 신나는]",61003,내적댄스 대박! 쉰나는 댄스곡,"[690285, 580860, 455945, 114387, 498452, 34758...",0,2019-12-21 00:09:24.000


## Preprocessing - remove rare songs, tags

In [187]:
cx = Counter()
for playlist in playlists:
    for song_id in playlist:
        cx[song_id] += 1
        
ct = Counter()
for tag in tags:
    for tag_comp in tag:
        ct[tag_comp] += 1

In [188]:
# 61만 곡, 2만9천여개 태그
print(len(cx), len(ct))

615142 29160


In [189]:
cx.most_common()

[(144663, 2174),
 (116573, 2121),
 (357367, 1981),
 (366786, 1919),
 (654757, 1647),
 (133143, 1633),
 (675115, 1598),
 (349492, 1586),
 (610933, 1538),
 (463173, 1531),
 (42155, 1527),
 (461341, 1526),
 (396828, 1525),
 (174749, 1512),
 (701557, 1507),
 (549178, 1500),
 (520093, 1495),
 (650494, 1479),
 (485155, 1479),
 (13281, 1472),
 (449244, 1468),
 (523521, 1463),
 (169984, 1462),
 (648628, 1457),
 (680366, 1454),
 (11657, 1452),
 (418935, 1443),
 (422915, 1442),
 (627363, 1424),
 (187047, 1424),
 (625875, 1421),
 (547967, 1421),
 (422077, 1419),
 (300087, 1415),
 (132994, 1414),
 (350309, 1412),
 (215411, 1395),
 (427724, 1394),
 (442014, 1390),
 (668128, 1389),
 (663256, 1375),
 (253755, 1375),
 (643628, 1375),
 (582252, 1367),
 (339802, 1364),
 (348200, 1363),
 (581799, 1363),
 (26083, 1355),
 (448116, 1355),
 (37748, 1353),
 (505036, 1334),
 (341513, 1325),
 (407828, 1324),
 (140867, 1323),
 (199262, 1320),
 (105140, 1311),
 (68348, 1311),
 (339124, 1310),
 (235773, 1307),
 (4

In [190]:
ct.most_common()

[('기분전환', 16207),
 ('감성', 11403),
 ('휴식', 11168),
 ('발라드', 10766),
 ('잔잔한', 10069),
 ('드라이브', 10042),
 ('힐링', 9782),
 ('사랑', 8662),
 ('새벽', 8320),
 ('밤', 7615),
 ('카페', 6825),
 ('추억', 6519),
 ('팝', 6425),
 ('힙합', 6412),
 ('이별', 5400),
 ('설렘', 4909),
 ('신나는', 4849),
 ('회상', 4578),
 ('매장음악', 4564),
 ('여행', 4529),
 ('인디', 4529),
 ('가을', 4041),
 ('락', 4007),
 ('겨울', 3937),
 ('스트레스', 3925),
 ('슬픔', 3527),
 ('뉴에이지', 3491),
 ('랩', 3420),
 ('봄', 3380),
 ('여름', 3295),
 ('댄스', 3290),
 ('Pop', 3247),
 ('재즈', 3213),
 ('알앤비', 3184),
 ('비오는날', 2982),
 ('까페', 2855),
 ('산책', 2731),
 ('운동', 2646),
 ('분위기', 2467),
 ('OST', 2323),
 ('일렉', 2234),
 ('클래식', 2153),
 ('피아노', 1920),
 ('위로', 1898),
 ('소울', 1869),
 ('클럽', 1644),
 ('팝송', 1623),
 ('연주곡', 1539),
 ('EDM', 1528),
 ('주말', 1490),
 ('트렌디', 1483),
 ('명곡', 1444),
 ('커피', 1355),
 ('취향저격', 1340),
 ('오후', 1338),
 ('아이돌', 1321),
 ('매장', 1311),
 ('저녁', 1286),
 ('어쿠스틱', 1241),
 ('그루브', 1237),
 ('아침', 1183),
 ('우울', 1133),
 ('편안한', 1093),
 ('RnB', 996),
 ('퇴근길',

In [191]:
len(Counter(el for el in ct.elements() if ct[el] <= 1))

17434

In [192]:
len(Counter(el for el in ct.elements() if ct[el] <= 2))

21379

In [193]:
len(Counter(el for el in ct.elements() if ct[el] <= 3))

23179

In [196]:
len(Counter(el for el in cx.elements() if cx[el] <= 5))

490525

In [197]:
len(Counter(el for el in cx.elements() if cx[el] <= 4))

470489

In [198]:
# 곡: 5회 이하, tag: 3회 이하 삭제
cx_ = Counter(el for el in cx.elements() if cx[el] > 5)
ct_ = Counter(el for el in ct.elements() if ct[el] > 3)

In [199]:
cx_.most_common()

[(144663, 2174),
 (116573, 2121),
 (357367, 1981),
 (366786, 1919),
 (654757, 1647),
 (133143, 1633),
 (675115, 1598),
 (349492, 1586),
 (610933, 1538),
 (463173, 1531),
 (42155, 1527),
 (461341, 1526),
 (396828, 1525),
 (174749, 1512),
 (701557, 1507),
 (549178, 1500),
 (520093, 1495),
 (650494, 1479),
 (485155, 1479),
 (13281, 1472),
 (449244, 1468),
 (523521, 1463),
 (169984, 1462),
 (648628, 1457),
 (680366, 1454),
 (11657, 1452),
 (418935, 1443),
 (422915, 1442),
 (627363, 1424),
 (187047, 1424),
 (625875, 1421),
 (547967, 1421),
 (422077, 1419),
 (300087, 1415),
 (132994, 1414),
 (350309, 1412),
 (215411, 1395),
 (427724, 1394),
 (442014, 1390),
 (668128, 1389),
 (663256, 1375),
 (253755, 1375),
 (643628, 1375),
 (582252, 1367),
 (339802, 1364),
 (348200, 1363),
 (581799, 1363),
 (26083, 1355),
 (448116, 1355),
 (37748, 1353),
 (505036, 1334),
 (341513, 1325),
 (407828, 1324),
 (140867, 1323),
 (199262, 1320),
 (105140, 1311),
 (68348, 1311),
 (339124, 1310),
 (235773, 1307),
 (4

In [200]:
ct_.most_common

<bound method Counter.most_common of Counter({'기분전환': 16207, '감성': 11403, '휴식': 11168, '발라드': 10766, '잔잔한': 10069, '드라이브': 10042, '힐링': 9782, '사랑': 8662, '새벽': 8320, '밤': 7615, '카페': 6825, '추억': 6519, '팝': 6425, '힙합': 6412, '이별': 5400, '설렘': 4909, '신나는': 4849, '회상': 4578, '매장음악': 4564, '여행': 4529, '인디': 4529, '가을': 4041, '락': 4007, '겨울': 3937, '스트레스': 3925, '슬픔': 3527, '뉴에이지': 3491, '랩': 3420, '봄': 3380, '여름': 3295, '댄스': 3290, 'Pop': 3247, '재즈': 3213, '알앤비': 3184, '비오는날': 2982, '까페': 2855, '산책': 2731, '운동': 2646, '분위기': 2467, 'OST': 2323, '일렉': 2234, '클래식': 2153, '피아노': 1920, '위로': 1898, '소울': 1869, '클럽': 1644, '팝송': 1623, '연주곡': 1539, 'EDM': 1528, '주말': 1490, '트렌디': 1483, '명곡': 1444, '커피': 1355, '취향저격': 1340, '오후': 1338, '아이돌': 1321, '매장': 1311, '저녁': 1286, '어쿠스틱': 1241, '그루브': 1237, '아침': 1183, '우울': 1133, '편안한': 1093, 'RnB': 996, '퇴근길': 993, '일렉트로니카': 986, '혼자': 976, '비': 973, '크리스마스': 948, '집중': 898, '가요': 883, '자장가': 873, '감각적인': 871, '새벽감성': 858, '센치': 830, '그리움': 811, '록': 795,

In [201]:
playlists_ = list(map(lambda x: [item for item in x if item in cx_], playlists))

In [223]:
len(playlists[400])

34

In [224]:
len(playlists_[400])

33

In [205]:
tags_ = list(map(lambda x: [item for item in x if item in ct_], tags))

In [210]:
tags[51]

['힙합', '감성', '눈물', '새벽', '랩', '슬픈랩', '그리움', '슬픔', '이별']

In [211]:
tags_[51]

['힙합', '감성', '눈물', '새벽', '랩', '그리움', '슬픔', '이별']

### song-tag matrix

In [225]:
list(product(tags_[0], playlists_[0]))

[('노동요', 159317),
 ('노동요', 368264),
 ('노동요', 274905),
 ('노동요', 643023),
 ('노동요', 452196),
 ('노동요', 424328),
 ('노동요', 380410),
 ('노동요', 280877),
 ('노동요', 248200),
 ('노동요', 204192),
 ('노동요', 520177),
 ('노동요', 409625),
 ('노동요', 274719),
 ('노동요', 258956),
 ('노동요', 342116),
 ('노동요', 438327),
 ('노동요', 481009),
 ('노동요', 621033),
 ('노동요', 382435),
 ('노동요', 212859),
 ('노동요', 37254),
 ('노동요', 526953),
 ('노동요', 502224),
 ('노동요', 144941),
 ('노동요', 217952),
 ('노동요', 431974),
 ('노동요', 488837),
 ('노동요', 94648),
 ('노동요', 384449),
 ('노동요', 386808),
 ('노동요', 313326),
 ('노동요', 191130),
 ('노동요', 443160),
 ('노동요', 660095),
 ('노동요', 406621),
 ('노동요', 474340),
 ('노동요', 46763),
 ('노동요', 247927),
 ('노동요', 1226),
 ('노동요', 37876),
 ('노동요', 346225),
 ('노동요', 530198),
 ('노동요', 650049),
 ('노동요', 65980),
 ('노동요', 660988),
 ('노동요', 423564),
 ('노동요', 62438),
 ('노동요', 139352),
 ('노동요', 243860),
 ('노동요', 699594),
 ('노동요', 50766),
 ('노동요', 695353),
 ('노동요', 675366),
 ('노동요', 124214),
 ('노동요', 565131),
 ('노동요', 627457),
 

In [226]:
# 태그의 경우, str type이기 때문에 index 부여 후 활용할 수 있는 t2i, i2t 만들어야
t2i, i2t = {}, {}
for i, t in enumerate(ct_.keys()):
    t2i[t] = i
    i2t[i] = t

In [274]:
t2i['발라드'], t2i['추억']

(59, 63)

In [227]:
len(t2i)

5981

In [254]:
cxt = Counter()
idx = 0
for slist, tlist in zip(tqdm(playlists_), tags_):
    pairs = list(product(slist, tlist))
    weight = like_cnts[idx]+1e-7
    for (x, t) in pairs:
        cxt[(x, t)] += weight
    idx += 1

In [248]:
for slist, tlist in zip(tqdm(playlists_), tags_):
    pairs = list(product(slist, tlist))
    for (x, t) in pairs:
        if x == 519391:
            print(t)

30대
이별
사랑
여름
발라드
추억회상
두근두근
새벽감성
차분한
멜랑콜리
피곤
이불
감성자극
감미로운
청춘
새벽감성
이별
이어폰
가을
감성
새벽
발라드
가요
설렘
사랑
카페
남자보컬
밤
인디
잔잔한
여행
가을
산책
숨은명곡
위로
힐링
어쿠스틱
설렘
3월
까페
설렘
사랑
겨울
감성자극
카페
겨울밤
새벽
포근한
추위
침대
따뜻한
찬바람
행복
결혼
연애
고백
설렘
사랑
멜로디좋은
산책
좋은노래
카페
사랑
인디
여름
휴식
주말
가을
여름밤
이불속
여행
바람
뒹굴뒹굴
썸남썸녀
설렘
달달한
나만알고싶은
봄
카페
오후
드라이브
봄날
힐링
결혼
설렘
사랑
노래방
노래
고백
커플
사랑
감성
가을
카페
저녁
가을밤
피곤
한강
기분전환
선선한
짝사랑
달달
고백
설렘
사랑
썸
러브
후회
감성
눈물
새벽
추억
아련한
그리움
보고픔
이별
기분전환
달달
행복
봄
사랑
휴식
비
카페
아이돌
댄스
발라드
여행
팝송
사랑
기분전환
봄
설렘
사랑
발라드
여름
사랑
행복
기분전환
데이트
이별
저녁
나만알고싶은
카페
지친하루
오후
드라이브
봄날
감성
감성
매장음악
봄
카페
잔잔한
새벽
힐링
사랑
숙면
밤
발라드
카페
아침
발라드
공원
여행
맑은
기분
산책
새벽
달달
봄바람
봄
설렘
벚꽃
기분전환
커플
너와나
사랑
달달한
겨울
매장음악
잔잔한
새벽
인디
따뜻한
까페
잔잔한
추억
새벽
저녁
밤
카페
드라이브
좋은노래
노을
밤새벽
감성
선선한
외로움
추천선곡
취향저격
오후
플레이리스트
랜덤
기분전환
장르불문
신나는
감성
카페
발라드
취향저격
기분전환
가요
팝송
취향
힐링
휴식
추억
회상
휴식
여유
가을
카페
달달
기분전환
사랑
여름
휴식
알앤비
댄스
발라드
드라이브
여행
기분전환
팝
팝
댄스
발라드
기분전환
알앤비
감성
카페
버스
추억
드라이브
봄날
상쾌한
힐링
센치
설렘
가을비
가을
가을밤
추억
쓸쓸한
감성
산책
이별
사랑
감성
봄
발라드
봄노래
드라이브
불금
위로
혼술
밤
신인
잔잔한
인디
그리움
감성
가을
잔잔한
갬성갬성
발라드
명곡
띵곡
인디
비오는날
휴식
감성자극
분위기깡패
새벽
새벽감성
잔잔

발라드
드라이브
기분전환
겨울노래
겨울감성
비오는날
감성
카페
발라드
드라이브
봄비
방콕
이불
감성
봄
카페
봄노래
달달
드라이브
봄캐롤
기분전환
편안한
가을감성
저녁
카페
가을밤
매장
밤새벽
발라드
저녁
카페
주말
봄노래
오후
드라이브
감성
봄
감성
봄
카페
봄노래
월요병
일상
드라이브
사랑
하트시그널
여름
한강
카페
주말
사랑
드라이브
감성
데이트
더위
겨울
감성
카페
발라드
잔잔한
기분전환
사랑
이별
감성발라드
이별
감성
가을
카페
발라드
가을밤
드라이브
불금
이별
솔로
감성
카페
봄노래
드라이브
집
가창력
커피
겨울
카페
주말
겨울밤
드라이브
연말
기분전환
크리스마스
겨울노래
추위
감성
카페
주말
봄노래
드라이브
나른한
밤
감성
카페
여유
여름밤
오후
드라이브
저녁
여름노래
겨울
감성
잔잔한
드라이브
발라드
행복
위로
기분
집중
감성
비
카페
여름밤
오후
매장
열대야
저녁
열대야
비오는날
감성
카페
오후
드라이브
불금
저녁
기분전환
센치한
이별
후회
저녁
주말
카페
발라드
가을밤
드라이브
커피
가을
야경
저녁
카페
여름밤
드라이브
감성
기분전환
여름노래
감성발라드
무더위
무드
분위기
밤
카페
지하철
드라이브
밤새벽
감성
퇴근길
감성
봄
카페
봄노래
드라이브
들썩들썩
봄캐롤
나들이
감성
봄
봄날
지칠때
멜랑콜리
밤새벽
사랑
밤
가을감성
한강
가을
카페
가을밤
드라이브
휴식
사랑
이별
감성발라드
여름
감성
감성자극
카페
주말
오후
드라이브
저녁
기분전환
감성
밤
카페
오후
드라이브
저녁
휴식
울고싶을때
기분전환
선선한
가을감성
저녁
카페
가을
발라드
오후
가을밤
드라이브
불금
이별
저녁
카페
발라드
가을밤
드라이브
불금
감성
감성발라드
이별
비오는날
여름
저녁
카페
오후
드라이브
센치
감성
기분전환
이별
카페
겨울
저녁
매장음악
카페
발라드
오후
겨울밤
드라이브
포근한
가을노래
저녁
카페
가을
여름밤
매장
발라드
감성
선선한
따스한
카페
주말
겨울밤
발라드
저녁
기분전환
겨울노래
추위
휴식
가을
카페
퇴근길
드라이브
기분전환
이별
센치한
추천곡
겨울
밤
발

휴식
연휴
발라드
추억에
잔잔한
슬픔
사랑
비오는날
한강
카페
오후
매장
드라이브
저녁
감성
여름
감성
카페
여유
주말
드라이브
기분전환
여름노래
감성음악
겨울
감성
캐럴
카페
월요병
드라이브
12월
겨울노래
추위
한강
가을
카페
여름밤
새벽
드라이브
감성
퇴근길
발라드
봄
카페
봄노래
달달
드라이브
벚꽃
연애
감미로운
저녁
밤
카페
취향저격
매장
드라이브
밤새벽
감성
음색
저녁
카페
주말
오후
가을밤
드라이브
독서
감성발라드
이별
감성
카페
봄바람
월요병
봄노래
청량
봄날
봄캐롤
외출
가을감성
저녁
카페
발라드
가을밤
드라이브
노을
서정적
쌀쌀한
감성
카페
가을
발라드
한강
드라이브
저녁
퇴근길
고막남친
저녁
카페
발라드
오후
드라이브
불금
낙엽
작업
추억
가을
카페
발라드
추운
가을밤
기분전환
사랑
이별
쌀쌀한
감성
매장음악
카페
겨울밤
드라이브
밤새벽
기분전환
추위
감성
가을
카페
발라드
가을밤
드라이브
커피
찬바람
쌀쌀한
겨울
매장음악
카페
겨울밤
밤새벽
기분전환
포근한
솔로
감성
카페
오후
드라이브
저녁
한강
기분전환
사랑
이별
분위기있는
감성
카페
봄노래
봄바람
드라이브
봄날
감성
밤
주말
드라이브
밤새벽
저녁
여름노래
선선한
겨울
휴식
카페
발라드
눈
연인
여행
사랑
이별
비오는날
여름
가을
발라드
추억
슬픔
기분전환
설렘
회상
숨은명곡
카페
발라드
취향저격
추억
겨울
기분전환
사랑
이별
인디
가을감성
저녁
가을
카페
발라드
가을밤
드라이브
밤새벽
퇴근길
겨울
저녁
매장음악
카페
오후
드라이브
기분전환
겨울노래
추위
감성
봄
카페
잔잔한
취향저격
드라이브
커피
기분전환
저녁
밤
카페
여름밤
매장
드라이브
감성
기분전환
바캉스
추억회상
연말
감성
카페
겨울밤
매장
드라이브
밤새벽
저녁
카페
여름밤
매장
드라이브
지칠때
그리움
감성
퇴근길
겨울
저녁
매장음악
겨울감성
카페
발라드
오후
드라이브
추위
가을노래
가을감성
저녁
카페
주말
발라드
오후
드라이브
감성
카페
잔잔한
드라이브
봄날
기분전환
사랑
이별
가을감성
감성
카페
발라드
가을밤
드라이브
밤새

비오는날
카페
가을
발라드
슬픔
기분전환
사랑
비오는날
감성
카페
주말
여름밤
차분한
저녁
기분전환
추억회상
저녁
매장음악
카페
연휴
취향저격
오후
사랑
이별
감성
가을
카페
가을밤
기분전환
사랑
이별
감성
카페
발라드
가을밤
드라이브
밤새벽
추석
노래방
겨울
감성
카페
잔잔한
발라드
드라이브
기분전환
사랑
감성
봄
카페
봄노래
발라드
드라이브
봄날
밤새벽
밤
감성
밤
카페
분위기
드라이브
불금
설렘
감성
밤
주말
발라드
봄노래
드라이브
밤새벽
잔잔한
밤
새벽
추억
슬픔
기분전환
설렘
이별
회상
발라드
가을감성
저녁
카페
발라드
가을밤
드라이브
불금
밤새벽
이별
감성
봄
카페
잔잔한
주말
드라이브
기분전환
사랑
신나는
비오는날
가을감성
카페
주말
드라이브
센치
기분전환
이별
감성발라드
겨울
가을
카페
발라드
드라이브
기분전환
사랑
이별
인디
한강
카페
여름밤
저녁
매장
드라이브
감성
기분전환
퇴근길
감성
잔잔한
분위기
드라이브
위로
기분전환
사랑
이별
여름
휴식
밤
잔잔한
슬픔
힐링
사랑
이별
까페
비
감성
가을밤
편안함
드라이브
밤새벽
저녁
발라드
감성
카페
여름밤
드라이브
노을
나른
저녁
기분전환
퇴근길
겨울
휴식
카페
발라드
기분전환
따뜻한
사랑
이별
인디
비오는날
분위기있는
저녁
카페
주말
오후
드라이브
기분전환
가을
겨울감성
저녁
카페
가을
발라드
가을밤
드라이브
밤새벽
감성
찬바람
가을노래
저녁
카페
주말
오후
매장
드라이브
피곤
휴식
가을감성
저녁
가을
카페
발라드
가을밤
이별
쌀쌀한
휴식
밤
가을
회상
추억
감성
사랑
이별
가을감성
저녁
카페
발라드
취향저격
오후
가을밤
드라이브
낙엽
어쿠스틱
카페
발라드
가을밤
밤새벽
사랑
감성발라드
이별
감성
카페
발라드
드라이브
봄노래
방콕
따스한
활기찬
집중
가을감성
저녁
카페
발라드
오후
가을밤
드라이브
퇴근길
쌀쌀한
겨울
저녁
매장음악
겨울감성
카페
겨울밤
취향저격
오후
매장
드라이브
여름
감성
카페
여름밤
매장
드라이브
불금
저녁
퇴근길
무더위
감성
노동요
카페
드라이브
월요병
방콕
집
감성
밤
잔잔

In [249]:
for slist, tlist in zip(tqdm(playlists_), tags_):
    pairs = list(product(slist, tlist))
    for (x, t) in pairs:
        if x == 640657:
            print(t)

감성힙합
유니크한
알앤비
기분전환
그루브한
소울
신나는
새벽감성
이별
이어폰
가을
이별
그리움
멜로디좋은
산책
좋은노래
카페
사랑
비오는날
겨울
감성
환희
겨울비
플라이투더스카이
추억소환
이별
노래방
노래
고백
커플
사랑
감성
가을
카페
저녁
가을밤
피곤
한강
기분전환
선선한
휴식
비
카페
아이돌
댄스
발라드
여행
팝송
사랑
기분전환
감성
슬픔
헤어짐
이별노래
이별
저녁
나만알고싶은
카페
지친하루
오후
드라이브
봄날
감성
카페
아침
발라드
공원
여행
맑은
기분
산책
새벽
기분전환
겨울
매장음악
잔잔한
새벽
인디
따뜻한
까페
저녁
밤
카페
드라이브
좋은노래
노을
밤새벽
감성
선선한
외로움
여름
휴식
알앤비
댄스
발라드
드라이브
여행
기분전환
팝
감성
봄
발라드
봄노래
드라이브
불금
위로
혼술
밤
감성
가을
잔잔한
갬성갬성
발라드
명곡
띵곡
인디
가을노래
가을감성
저녁
카페
가을
오후
사랑
집중
데이트
감성발라드
치유
힐링
발라드
소울
회상
비오는날
가을
카페
가을밤
여행
기분전환
사랑
스트레스
비
겨울
감성
카페
발라드
추억
기분전환
사랑
이별
인디
여름
감성
카페
여름밤
잔잔한
드라이브
노을
기분전환
외출
사랑
감성
카페
주말
드라이브
봄날
벚꽃
설렘
사랑
이별
봄
감성
봄
카페
취향저격
봄비
매장
드라이브
봄날
기분전환
잔잔한노래
무드
감성
밤
새벽
새벽감성
끈적한노래
여름
저녁
노동요
카페
취향저격
오후
드라이브
감성
기분전환
여름노래
감성
주말
잔잔한
꿀성대
휴식
기분전환
꿀보이스
꿀맛
고막여친
따뜻한
밤
발라드
새벽
추천곡
잠들기전
좋아요
신인
역주행
인디
미세먼지
감성
봄캐롤
카페
청량
휴식
사랑
힐링
봄
감성
봄
카페
봄노래
봄바람
드라이브
밤새벽
Chill
가을감성
저녁
가을
카페
가을밤
오후
매장
드라이브
노을
감성발라드
감성
봄
카페
우울한날
매장
드라이브
벚꽃
연애
사랑
이별
감성
봄
카페
오후
저녁
봄캐롤
사랑
분위기깡패
쌀쌀한
저녁
밤
카페
매장
봄날
밤새벽
감성
사랑
이별
겨울
카페
주말
발라드
매장
드라이브
겨울노래
로맨틱
달콤한
겨울

가을노래
저녁
주말
카페
발라드
오후
드라이브
감성
가을
휴일
감성
카페
주말
발라드
매력
드라이브
겨울
겨울노래
가을노래
가을감성
고독
주말
발라드
가을밤
드라이브
밤새벽
감성
연말
감성
카페
겨울밤
드라이브
밤새벽
12월
추위
감성
새벽
그리움
슬픔
사랑
이별
여름
저녁
카페
주말
브런치
오후
드라이브
감성
여름휴가
휴일
저녁
가을
카페
발라드
오후
드라이브
감성
이별
쌀쌀한
가을노래
가을감성
저녁
카페
가을밤
드라이브
불금
감성발라드
이별
그리움
가을노래
가을감성
저녁
가을
카페
발라드
오후
드라이브
낙엽
쌀쌀한
가을감성
카페
주말
발라드
드라이브
가을
이별
감성발라드
저녁
카페
여름밤
매장
감성
여름노래
바캉스
퇴근길
비오는날
감성
카페
잔잔한
발라드
드라이브
위로
기분전환
사랑
인디
감각적인
겨울
저녁
노동요
카페
발라드
오후
드라이브
기분전환
디즈니
겨울
감성
카페
발라드
추억
설렘
사랑
인디
겨울
감성
카페
오후
드라이브
불금
저녁
겨울노래
사랑
이별
감성
장마
카페
매장
드라이브
흐린날
기분전환
이별
센치한
매장음악
카페
잔잔한
발라드
커피
설렘
사랑
여름
저녁
카페
취향저격
오후
매장
드라이브
감성
기분전환
여름휴가
저녁
매장음악
카페
주말
겨울밤
드라이브
감성
노래방
감성
주말
오후
드라이브
저녁
기분전환
여름노래
더위
선선한
감성
봄
카페
잔잔한
추억
힐링
기분전환
사랑
이별
발라드
가을노래
저녁
노동요
카페
가을
발라드
월요병
드라이브
감성
오후
휴일
감성
카페
오후
드라이브
여름
휴식
저녁
기분전환
비오는날
비오는거리
비오는날
저녁
카페
여름밤
매장
드라이브
감성
기분전환
감성발라드
비오는날
여름
저녁
장마
카페
오후
드라이브
불금
감성
센치한
겨울
감성
카페
보컬
기분전환
사랑
이별
감성
카페
봄노래
인디팝
설레임
드라이브
봄날
벚꽃
사랑
커피숍
가을감성
흐린날
카페
주말
가을비
드라이브
감성발라드
이별
가을감성
저녁
카페
주말
발라드
오후
드라이브
가을
쌀쌀한
가을감성
저녁
카페
주말
발라드
드라이브
여행
맑은
겨울
감성
카페
발라드

겨울
저녁
매장음악
카페
발라드
오후
겨울밤
드라이브
추위
숨은명곡
감성
카페
발라드
취향저격
새벽
겨울
기분전환
인디
카페
겨울밤
추억
드라이브
위로
기분전환
사랑
발라드
인디
감성
봄
카페
발라드
드라이브
불금
봄캐롤
밤
여름
저녁
노동요
카페
오후
드라이브
감성
기분전환
여름노래
비오는날
감성
밤
카페
드라이브
불금
저녁
기분전환
저녁
매장음악
카페
연휴
겨울밤
발라드
드라이브
설날
감성
카페
주말
잔잔한
분위기
위로
슬픔
기분전환
사랑
숨은명곡
카페
발라드
취향저격
드라이브
겨울
기분전환
사랑
인디
비오는날
저녁
봄
카페
고독
눈
오후
주말
드라이브
서정적인
연말
휴식
발라드
드라이브
겨울
음색
사랑
이별
추위
인디
감성
시원한노래
카페
여름밤
오후
매장
드라이브
저녁
기분전환
무더위
겨울
휴식
매장음악
카페
발라드
추억
슬픔
기분전환
산책
까페
겨울
저녁
카페
발라드
오후
드라이브
12월
겨울노래
따뜻한
매장음악
밤
새벽
추억
슬픔
설렘
이별
비오는날
감성
발라드
잔잔한
센치
기분전환
휴식
매장음악
카페
발라드
잔잔한
커피
기분전환
휴식
노동요
로맨틱
카페
발라드
드라이브
감성
꿀보이스
감성
카페
겨울밤
드라이브
밤새벽
불금
저녁
구름
감성
비
카페
낭만
rain
사랑
이별
쓸쓸함
빗방울
감성
봄
카페
봄노래
발라드
드라이브
일상
공부
밤
비오는날
솔로
감성
카페
여름밤
드라이브
기분전환
퇴근길
겨울
카페
발라드
버스
추억
드라이브
사랑
이별
인디
겨울
감성
카페
발라드
드라이브
기분전환
사랑
이별
집중
감성
밤
카페
오후
드라이브
센치
불금
저녁
이별
가을노래
저녁
카페
발라드
감성
오후
매장
드라이브
휴식
가을밤
겨울
저녁
매장음악
카페
발라드
취향저격
오후
겨울밤
드라이브
감성
카페
여름밤
취향저격
소나기
드라이브
저녁
기분전환
여름노래
퇴근길
비오는날
흐린날
노동요
로맨틱
카페
라떼
감성
꿀보이스
감성
밤
회상
발라드
설렘
이별
퇴근길
감성
주말
카페
잔잔한
한강
드라이브
저녁
밤
감성
봄
잔잔한
분위기
봄캐롤
사랑
이별
발라드
쌀쌀한
비오는날

매장음악
가을
커피
힐링
기분전환
설렘
까페
겨울
카페
주말
겨울밤
발라드
드라이브
기분전환
추위
찬바람
휴일
감성
봄
봄노래
드라이브
분위기
방콕
밤
겨울
휴식
발라드
기분전환
사랑
이별
추위
겨울
저녁
겨울감성
카페
겨울밤
드라이브
추위
추천곡
겨울감성
잔잔한
추억
드라이브
그리움
위로
기분전환
사랑
감성
카페
겨울밤
발라드
드라이브
밤새벽
추위
솔로
감성
봄
봄비
분위기
기분전환
사랑
이별
우울할때
감성
밤
카페
발라드
드라이브
힙한
겨울
감성
카페
잔잔한
취향저격
위로
기분전환
사랑
이별
공부
카페
주말
겨울밤
드라이브
기분전환
추위
따뜻한
겨울
따스한
매장음악
카페
겨울밤
드라이브
밤새벽
따뜻한
겨울
감성
카페
발라드
추억
드라이브
사랑
이별
겨울
감성
매장음악
카페
발라드
겨울밤
드라이브
감미로운
감성
잔잔한
발라드
센치
기분전환
사랑
인디
감성
카페
주말
봄노래
드라이브
벚꽃
기분전환
사랑
봄
감성
봄
카페
발라드
낭만
우울
잔잔한
기분전환
사랑
감성
봄
카페
발라드
멜랑콜리
밤새벽
고막여친
밤
감성
노동요
카페
오후
드라이브
저녁
기분전환
겨울노래
감성
밤
주말
발라드
봄노래
드라이브
봄캐롤
여름
휴식
카페
가을
새벽
잔잔한
힐링
설렘
산책
밤
겨울
감성
카페
잔잔한
발라드
드라이브
기억
기분전환
사랑
인디
휴식
매장음악
가을
카페
잔잔한
힐링
사랑
이별
감성
카페
겨울밤
발라드
드라이브
커피
기분전환
겨울
감성
카페
잔잔한
발라드
드라이브
기분전환
사랑
이별
겨울
저녁
카페
겨울밤
취향저격
드라이브
꿀음색
겨울노래
잔잔한
취향저격
추억
드라이브
위로
기분전환
사랑
이별
인디
겨울
잔잔한
추억
슬픔
사랑
이별
회상
겨울
감성
카페
발라드
드라이브
기분전환
사랑
이별
인디
겨울
감성
발라드
추억
위로
기분전환
사랑
분위기
감성
카페
오후
드라이브
저녁
기분전환
사랑
여름노래
이별
저녁
카페
로맨틱
봄노래
취향저격
오후
드라이브
감성
기분전환
감성
매장음악
카페
봄노래
취향저격
발라드
드라이브
기분전환
겨울
감성
노동요
겨울감성
카페
오후
드라이브


In [255]:
data, rows, cols = [], [], []
s2t_samples = Counter()

for (x, t), n in tqdm(cxt.items()):
    rows.append(x)
    cols.append(t2i[t])
    data.append(n)
    s2t_samples[(x, t)] = data[-1]

In [256]:
s2t = csr_matrix((data, (rows, cols)))

In [257]:
print('%d non-zero elements' % s2t.count_nonzero())
print('Sample s2t values\n', pformat(s2t_samples.most_common()[:10]))

6750205 non-zero elements
Sample s2t values
 [((580074, '발라드'), 135061.00004220032),
 ((140867, '팝'), 127851.00003010023),
 ((173943, '발라드'), 125469.00004190032),
 ((113958, '추억'), 118691.00002020015),
 ((222763, '발라드'), 117628.00002990023),
 ((1236, '추억'), 116929.00002100016),
 ((117595, '발라드'), 116339.00005080039),
 ((443160, '기분전환'), 112973.00000290002),
 ((635537, '발라드'), 110700.00003300025),
 ((484234, '팝'), 109656.0000125001)]


In [258]:
s2t_dense = s2t.todense()

In [267]:
s1 = np.array(s2t_dense[580074]).reshape(-1,)
s2 = np.array(s2t_dense[173943]).reshape(-1,)

In [260]:
s3 = np.array(s2t_dense[159327]).reshape(-1,)

In [277]:
t1 = np.array(s2t_dense[:,59]).reshape(-1,)
t2 = np.array(s2t_dense[:,63]).reshape(-1,)

In [268]:
def cos_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    l2_norm = (np.sqrt(sum(np.square(v1))) * np.sqrt(sum(np.square(v2))))
    similarity = dot_product / l2_norm     
    
    return similarity

In [269]:
cos_similarity(s1, s2)

0.951279284131902

In [263]:
cos_similarity(s1, s3)

0.4569902439171708

In [278]:
cos_similarity(t1, t2)

0.724387573754949

In [270]:
songs[songs['id']==580074]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
580074,"[GN0101, GN0103]",19990100,A Night In Seoul,5422,[6017],여전히 아름다운지,[GN0100],[김연우],580074


In [271]:
songs[songs['id']==173943]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
173943,"[GN0104, GN0401]",20040329,Can U See The Bright,34289,[8530],...사랑했잖아...,"[GN0400, GN0100]",[린],173943


In [272]:
songs[songs['id']==159327]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
159327,[GN0101],20010619,First Story,6846,[10575],눈물에 얼굴을 묻는다,[GN0100],[장나라],159327


In [282]:
save_npz('../data/s2t_matrix.npz', s2t)